# Real-world Example

Resource: https://web.itu.edu.tr/topcuil/ya/SEN301previousexamquestions.pdf


A company is involved in the production of two items (X and Y). The resources need to produce X and Y are twofold, namely machine time for automatic processing and craftsman time for hand finishing. The table below gives the number of minutes required for each item

| Item | Machine time |  Craftsman time |
| - | - | - |
| Item X | 13 | 20 |
| Item Y | 19 | 29 |


The company has 40 hours of machine time available in the next working week but only 35 hours of craftsman time. Machine time is costed at £10 per hour worked and craftsman time is costed at £2 per hour worked. Both machine and craftsman idle times incur no costs. The revenue received for each item produced (all production is sold) is £20 for X and £30 for Y. The company has a specific contract to produce 10 items of X per week for a particular customer. Formulate the problem of deciding how much to produce per week as a linear program. 

In [122]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("SAT")
if not solver:
  raise RuntimeError('Could not create solver')

x = solver.IntVar(10, solver.infinity(), "x")
y = solver.IntVar(0, solver.infinity(), "y")

print("Number of variables =", solver.NumVariables())

Number of variables = 2


In [123]:
# Machine time constraint.
solver.Add(13 * x + 19 * y <= 40 * 60)

# Craftsman time constraint.
solver.Add(20 * x + 29 * y <= 35 * 60)

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 2


In [124]:
def cost_per_hour(*, minutes, cost):
  return minutes / 60 * cost

# Objective function.
solver.Maximize(
  x * (20 - cost_per_hour(minutes=13, cost=10) + cost_per_hour(minutes=20, cost=2)) + 
  y * (30 - cost_per_hour(minutes=19, cost=10) + cost_per_hour(minutes=29, cost=2))
)

In [125]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

Solving with CP-SAT solver v9.8.3296


In [126]:
if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print(f"Objective value = {solver.Objective().Value():0.1f}")
    print(f"x = {x.solution_value():0.1f} machine_hours = {x.solution_value() * 13 / 60:0.1f} craftsman_hours = {x.solution_value() * 20 / 60:0.1f}")
    print(f"y = {y.solution_value():0.1f} machine_hours = {y.solution_value() * 19 / 60:0.1f} craftsman_hours = {y.solution_value() * 29 / 60:0.1f}")
else:
    print("The problem does not have an optimal solution.")

Solution:
Objective value = 2001.2
x = 12.0 machine_hours = 2.6 craftsman_hours = 4.0
y = 64.0 machine_hours = 20.3 craftsman_hours = 30.9


When `y = 0`:

```
Solution:
Objective value = 1942.5
x = 105.0 machine_hours = 22.8 craftsman_hours = 35.0
y = 0.0 machine_hours = 0.0 craftsman_hours = 0.0
```


When `x = 0`:

```
Solution:
Objective value = 2001.6
x = 0.0 machine_hours = 0.0 craftsman_hours = 0.0
y = 72.0 machine_hours = 22.8 craftsman_hours = 34.8
```

Optimal without the min `x = 10` constraint:

```
Solution:
Objective value = 2010.8
x = 2.0 machine_hours = 0.4 craftsman_hours = 0.7
y = 71.0 machine_hours = 22.5 craftsman_hours = 34.3
```